In [1]:
import numpy
import random
import matplotlib.pyplot as plt
import math

In [2]:
# TSP 데이터 읽어오기
def getCity():
    cities = []
    f = open('TSP51.txt')
    for i in f.readlines():
        node_city_val = i.split()
        cities.append(
            [node_city_val[0], float(node_city_val[1]), float(node_city_val[2]) ] # 노드 정보, x, y
        )

In [4]:
#주어진 염색체 요소의 좌표 -> 거리
def calcDistance(cities):
    total_sum = 0
    for i in range(len(cities) - 1):
        cityA = cities[i] 
        cityB = cities[i + 1]

        # 거리 계산
        d = math.sqrt(
            math.pow(cityB[1]-cityA[1], 2) + math.pow(cityB[2]- cityA[2], 2) 
        )
        
        total_sum += d

    cityA = cities[0]
    cityB = cities[-1]
    d = math.sqrt(
        math.pow(cityB[1]-cityA[1], 2) + math.pow(cityB[2]- cityA[2], 2) 
        )
    
    total_sum += d

    return total_sum
    

In [5]:
# selecting the population
def selectPopulation(cities, size):
    population = []

    for i in range(size):
        c = cities.copy()
        random.shuffle(c)
        distance = calcDistance(c)
        population.append([distance, c])

    fitest = sorted(population)[0]

    return population, fitest

In [ ]:
# genetic algorithm
def geneticAlgorithm(
        population,
        lenCities,
        TOURNAMENT_SELECTION_SIZE,
        MUTATION_RATE, #돌연변이 비율
        CROSSOVER_RATE, # 교차 division point
        TARGET,
        ):
    gen_number = 0

    for i in range(200):
        new_population = []

        #가장 적합도가 높은 염색체 두 개 -> 다음 세대 부모 염색체(해당 부분에선 거리기준 적합도 평가)
        new_population.append(sorted(population)[0])
        new_population.append(sorted(population)[1])

        for i in range(int((len(population) - 2) / 2)): #division point -> 중간지점
            #CROSSOVER
            random_number = random.random()
            if random_number < CROSSOVER_RATE:
                parent_chromosome1 = sorted(
                    random.choices(population, k = TOURNAMENT_SELECTION_SIZE)
                )[0]

                parent_chromosome2 = sorted(
                    random.choices(population, k = TOURNAMENT_SELECTION_SIZE)
                )[0]

                point = random.randint(0, lenCities-1)

                child_chromosome1 = parent_chromosome1[1][0:point]
                
                # TSP 문제 특성상 서로 교차 하여 자손을 만들 때 겹치면 안 됨
                for j in parent_chromosome2[1]: 
                    if (j in child_chromosome1) == False:
                        child_chromosome1.append(j)

                child_chromosome2 = parent_chromosome2[1][0:point]
                for j in parent_chromosome1[1]: 
                    if (j in child_chromosome2) == False:
                        child_chromosome2.append(j)

            # crossover not happen

            else:
                child_chromosome1 = random.choices(population)[0][1]
                child_chromosome2 = random.choices[population][0][1]
            
            #MUTATION
            # 난수가 mutation 확률보다 낮으면 염색체의 요소 서로 변경
            if random.random() < MUTATION_RATE:
                point1 = random.randint(0, lenCities - 1)
                point2 = random.randint(0, lenCities - 1)
                child_chromosome1[point1], child_chromosome1[point2] = (
                    child_chromosome1[point2],
                    child_chromosome1[point1]
                )

                point1 = random.randint(0, lenCities - 1)
                point2 = random.randint(0, lenCities - 1)
                child_chromosome2[point1], child_chromosome2[point2] = (
                    child_chromosome2[point2],
                    child_chromosome2[point1]
                )

            new_population.append([calcDistance(child_chromosome1), child_chromosome1])
            new_population.append([calcDistance(child_chromosome2), child_chromosome2])

        population = new_population

        gen_number+=1

        if gen_number % 10 ==0:
            print(gen_number, sorted(population)[0][0])

        if sorted(population)[0][0] < TARGET:
            break
    answer = sorted(population)[0]

    return answer, gen_number


In [3]:
# map 시각화
def drawMap(city, answer):
    for j in city:
        plt.plot(j[1], j[2], "ro")
        plt.annotate(j[0], (j[1], j[2]))

    for i in range(len(answer[1])):
        try:
            first = answer[1][i]
            second = answer[1][i+1]

            plt.plot([first[1], second[1]], [first[2], second[2]], 'gray')
        except:
            continue

        first = answer[1][0]
        second = answer[1][-1]
        plt.plot([first[1], second[1]], [first[2], second[2]], 'gray')

        plt.show()

In [ ]:
def main():
    #initial values
    POPULATION_SIZE = 2000
    TOURNAMENT_SELECTION_SIZE = 4
    MUTATION_RATE = 0.1
    CROSSOVER_RATE = 0.9
    TARGET = 450.0

    cities = getCity()
    firstPopulatiom, firstFitest = selectPopulation(cities, POPULATION_SIZE)
    answer, genNumber = geneticAlgorithm(
        firstPopulatiom,
        len(cities),
        TOURNAMENT_SELECTION_SIZE,
        MUTATION_RATE,
        CROSSOVER_RATE,
        TARGET,
        
    )